In [1]:
# Import General Libraries 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning Libraries
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Privacy Package
from privacy.analysis.rdp_accountant import compute_rdp
from privacy.analysis.rdp_accountant import get_privacy_spent
from privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

Using TensorFlow backend.
W0726 12:12:08.830678 140625388181312 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0726 12:12:08.834008 140625388181312 deprecation_wrapper.py:119] From /home/thomas/Downloads/privacy/privacy/optimizers/dp_optimizer.py:230: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0726 12:12:08.834610 140625388181312 deprecation_wrapper.py:119] From /home/thomas/Downloads/privacy/privacy/optimizers/dp_optimizer.py:231: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0726 12:12:08.835071 140625388181312 deprecation_wrapper.py

In [2]:
# Parameters
dpsgd = True
learning_rate = 0.25
noise_multipliers = [.71, 1.8]
l2_norm_clip = .05
batch_size = 256
epochs = 3
microbatches = 1
model_dir = None

In [3]:
def data_processing():
    data = pd.read_csv('datasets/hmda_GAVANCSC_clean.csv')
    
    dataevensamp = pd.concat([(data[data["action_taken_name"] == 1]).sample(n=len(data[data["action_taken_name"]==0])),
                          data[data["action_taken_name"]==0]])
    data = dataevensamp.sample(frac=1)

    target = data['action_taken_name']
    target = to_categorical(target) 
    predictors = data.drop(['action_taken_name'], axis=1)
    predictors = predictors.drop(predictors.columns[0], axis=1)
    
    return predictors, target

In [4]:
def compute_epsilon(steps, noise_multiplier):
  """Computes epsilon value for given hyperparameters."""
  if noise_multiplier == 0.0:
    return float('inf')
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = batch_size / predictors.shape[0]
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=steps,
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 60000 training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]

In [5]:
predictors, target = data_processing()

n_cols = predictors.shape[1]
optimizers = []
for noise in noise_multipliers:
    optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=l2_norm_clip,
        noise_multiplier=noise,
        num_microbatches=microbatches,
        learning_rate=learning_rate,
        unroll_microbatches=True)
    optimizers.append(optimizer)

x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.20, random_state = 0)

In [6]:
import lime
import lime.lime_tabular
feature_names=list(x_train.columns)
categorical_indices=[]
for x in range(0, len(feature_names)):
    if("name" in feature_names[x] or "abbr" in feature_names[x]):
        categorical_indices.append(x)
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=feature_names, categorical_features=categorical_indices, verbose=False, discretize_continuous=True, discretizer='decile')

In [16]:
def rankingFn(row, predict_proba, rankingsDict):
    exp = explainer.explain_instance(row, predict_proba, num_features=x_test.shape[1])
    explist = exp.as_list()
    for x in range(0,topN):
        item = explist[x][0]
        key = item.split('=')[0]
        if ' < ' in key:
            key = key.split(' < ')[1]
        if ' ' in key:
            key = key.split(' ')[0]
        rankingsDict[key][x] += 1
        rankingsDict[key][topN].append(explist[x][1])

In [8]:
privatemodels = []
nummodels = 5
for optimizer in optimizers:
    model_of_ep =[]
    for x in range(0,nummodels):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape = (n_cols,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(6, activation='relu'),
            tf.keras.layers.Dense(2, activation='softmax'),
        ])

        model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

        model_of_ep.append(model)
    privatemodels.append(model_of_ep)

W0726 12:12:13.313097 140625388181312 deprecation.py:506] From /home/thomas/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
for epsilon in privatemodels:
    for m in epsilon:
        m.fit(x_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose = 0)
        scores = m.evaluate(x_test, y_test, verbose=0)
        print("%s: %.2f%%" % (m.metrics_names[1], scores[1]*100))

W0726 12:12:14.653934 140625388181312 deprecation.py:323] From /home/thomas/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/clip_ops.py:286: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0726 12:12:14.676153 140625388181312 deprecation_wrapper.py:119] From /home/thomas/Downloads/privacy/privacy/dp_query/gaussian_query.py:101: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



acc: 67.95%
acc: 68.84%
acc: 66.62%
acc: 68.74%
acc: 67.73%
acc: 51.47%
acc: 49.99%
acc: 52.26%
acc: 62.53%
acc: 49.54%


In [10]:
nonprivatemodels = []
for x in range(0,nummodels):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape = (n_cols,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax'),
    ])

    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

    nonprivatemodels.append(model)

In [11]:
for m in nonprivatemodels:
    m.fit(x_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose = 0)
    scores = m.evaluate(x_test, y_test, verbose=0)
    print("%s: %.2f%%" % (m.metrics_names[1], scores[1]*100))

acc: 71.02%
acc: 71.15%
acc: 70.71%
acc: 71.45%
acc: 71.33%


In [17]:
all_models = [nonprivatemodels]
for set_of_models in privatemodels:
    all_models.append(set_of_models)
num_samples = 5
test_data = x_test.sample(n=num_samples)

In [18]:
all_rankings = []
topN = 5
for models in all_models:
    type_rankings = []
    for m in models:
        ranking ={}
        for x in feature_names:
            ranking[x] = []
            for y in range(0,topN):
                ranking[x].append(0)
            ranking[x].append([])
        test_data.apply(rankingFn, axis=1, args=[m.predict_proba, ranking])
        type_rankings.append(ranking)
    all_rankings.append(type_rankings)

In [14]:
colored_names = {
    'tract_to_msamd_income' : 'black',
    'population' : 'black',
    'minority_population' : 'black',
    'number_of_owner_occupied_units' : 'black',
    'number_of_1_to_4_family_units' : 'black' ,
    'loan_amount_000s' : 'green',
    'hud_median_family_income' : 'black',
    'applicant_income_000s' : 'green',
    'property_type_name' : 'green',
    'owner_occupancy_name' : 'green',
    'loan_type_name' : 'green',
    'loan_purpose_name' : 'green',
    'lien_status_name' : 'green',
    'co_applicant_sex_name' : 'green',
    'applicant_sex_name' : 'red',
    'applicant_race_name_1' : 'red',
    'applicant_ethnicity_name' : 'red',
    'agency_abbr' : 'black'
}
renamed_features = {
    'tract_to_msamd_income' : 'Tract Income Ratio',
    'population' : 'Population',
    'minority_population' : 'Tract % Minority',
    'number_of_owner_occupied_units' : 'Tract # Owners',
    'number_of_1_to_4_family_units' : 'Tract # Families' ,
    'loan_amount_000s' : 'Loan Amount',
    'hud_median_family_income' : 'Tract Income',
    'applicant_income_000s' : 'App. Income',
    'property_type_name' : 'Property Type',
    'owner_occupancy_name' : 'Owner Occupancy',
    'loan_type_name' : 'Loan Type',
    'loan_purpose_name' : 'Loan Purpose',
    'lien_status_name' : 'Lien Status',
    'co_applicant_sex_name' : 'Has Co-app',
    'applicant_sex_name' : 'App. Sex',
    'applicant_race_name_1' : 'App. Race',
    'applicant_ethnicity_name' : 'App. Ethnicity',
    'agency_abbr' : 'Agency'
}
colored_bars = [
    (208/255,209/255,230/255),
    (166/255,189/255,219/255),
    (116/255,169/255,207/255),
    (43/255,140/255,190/255),
    (4/255,90/255,141/255)
]
colored_bars.reverse()

In [27]:
fig = plt.figure(figsize=(6*(len(all_models)), 25),dpi=400)
gs = plt.GridSpec(len(all_models[0]),len(all_models))
#plt.subplots_adjust(right=.8)
#plt.figtext(.83, .54, 'Feature Color Legend', color='black', fontweight='bold', size=14)
#plt.figtext(.83, .52, 'Applicant Controlled Info', color='green', fontweight='bold')
#plt.figtext(.83, .50, 'Applicant Uncontrolled Info', color='blue', fontweight='bold')
#plt.figtext(.83, .48, 'Protected Classes', color='red', fontweight='bold')
#plt.figtext(.83, .46, 'Other', color='black', fontweight='bold')

#fig.patches.extend([plt.Rectangle((0.82,0.45),0.18,0.1,
#                                  fill=False, color='black', alpha=1, zorder=1000,
#                                  transform=fig.transFigure, figure=fig)])


plots = []
for row in range(len(all_models)):
    plots.append([])
    for col in range(len(all_models[0])):
        plots[row].append(fig.add_subplot(gs[col,row]))
plt.subplots_adjust(wspace=.1)

#Features we will show (Determine by using the first nonprivate model)
important_features =[]
ranking = all_rankings[0][0]
d = sorted(ranking.items(), key=lambda x:(np.sum(x[1][:-1]),x[1][0],x[1][1],x[1][2],x[1][3],x[1][4]))
for item in d:
    if np.sum(item[1][:-1]) != 0:
        important_features.append(item[0])
ind = np.arange(len(important_features))
width = .35

for row in range(len(plots)):
    for col in range(len(plots[row])):
        ranking = all_rankings[row][col]
        #Establishes pltdata
        pltdata = []
        for x in range(0,topN):
            pltdata.append([])
            for y in range(0,len(important_features)):
                pltdata[x].append(0)
        for x in range(len(important_features)):
            for y in range(topN):
                pltdata[y][x] += ranking[important_features[x]][y]
        
    
    
        for index in range(0,len(pltdata)):
            index = len(pltdata) - index - 1
            info = tuple(pltdata[index])
            offset = np.zeros(len(pltdata[0]))
            for x in range(index+1,len(pltdata)):
                for y in range(0, len(pltdata[x])):
                    offset[y] += pltdata[x][y]
            offset = tuple(offset)
            plots[row][col].barh(ind, info, width, left=offset, label=(index+1), color=colored_bars[index])
        labels = important_features.copy()
        if col == 0:
            if row == 0:
                plots[row][col].set_title('Non-private Models', fontweight='bold', size=16)
            else:
                plots[row][col].set_title(('Private Models of ε-' + (str)(round(compute_epsilon(epochs * predictors.shape[0] // batch_size, noise_multipliers[row-1]), 2))), fontweight='bold', size=16)
        #plots[row][col].set_title('Private Model ' + str(i+1))
        plots[row][col].set_xticks(np.arange(0, num_samples+1, num_samples/5))
        if row == 0:
            plots[row][col].set_yticks(ind)
            pltlabels = plots[row][col].set_yticklabels(labels)
            [i.set_color(colored_names[i.get_text()]) for i in pltlabels]
            for x in range(len(labels)):
                labels[x] = renamed_features[labels[x]]
            pltlabels = plots[row][col].set_yticklabels(labels)
        else:
            plots[row][col].set_yticks([])
        handles, labels = plots[row][col].get_legend_handles_labels()
        if row == len(plots) - 1:
            plots[row][col].legend(handles[::-1], labels[::-1], title='Rank', loc='center left', bbox_to_anchor=(1, 0.5))
        plots[row][col].set_xlabel('# of times ranked top 5 feature')
    
plt.savefig('all_model_valued_features_ep_' + ",".join([str(round(compute_epsilon(epochs * predictors.shape[0] // batch_size, noise), 2)) for noise in noise_multipliers]) + '.pdf', bbox_inches='tight')
plt.show()